# Aufbauch einer Suchmachine für Bundestagsreden mit SkitLearn <div class="tocSkip">

## Motivation: 
Das Ziel dieses Notebooks ist es, eine Suchmachine aufzubauen, mit der Datenanalysen auf eine spezifische Gruppe von Reden durchzuführen sind. 

## Hilfsfunktionen und Einstellungen 


In [43]:
%run "../resources/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# path to import blueprints packages
sys.path.append('./packages')

from sklearn.feature_extraction.text import CountVectorizer


## Initialisierung
Einlesen der Daten aus der Datenbank 

In [44]:
database = '../resources/bt_reden_final.sqlite'

sql = """
SELECT * from bt_reden_prepared
"""

con = sqlite3.connect(database)
df = pd.read_sql_query(sql, con)
con.close()

In [45]:
count_vect = CountVectorizer(lowercase=False, min_df=0, max_df=0.8)

X_tf = count_vect.fit_transform(df['text'])

print("%d x %d Count-Matrix (= BoW = TF)" % X_tf.shape)

24647 x 199247 Count-Matrix (= BoW = TF)


In [46]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

class SearchEngine:
    
    def __init__(self, vectorizer, df, index_col):
        """Constructor. 
        
        Arguments: 
        - vectorizer: class implementing fit_transform
        - df: data frame with documents
        - index_col: text column to be indexed"""
        
        self.count_vect = vectorizer
        self.df = df
        self.index_col = index_col
        self.X = self.count_vect.fit_transform(self.df[index_col])

        
    def search(self, query, max_results=10):
        """Ranks documents based on query."""
        
        query_v = self.count_vect.transform([query])
        
        sims = cosine_similarity(self.X, query_v)
        sims = np.ndarray.flatten(sims) # unnest matrix into array

        self.df['sim'] = sims
        return self.df.sort_values(by=['sim'], ascending=False).head(max_results)

Die min_df, max_df werden so gewählt, dass generische Wörter ausgefiltert werden. 

In [47]:
engine = SearchEngine(TfidfVectorizer(min_df=0, max_df=0.8), df, index_col='clean_text')

Wir definieren eine Menge an Schlagwörter und erstellen ein Balkendiagramm, das zeigt, welche Parteien mit welcher Häufigkeit über ein bestimmtes Thema sprechen

In [75]:
pd.options.plotting.backend = "plotly"
parteien = ['AfD', 'SPD', 'BÜNDNIS 90/DIE GRÜNEN', 'CDU/CSU', 'DIE LINKE','FDP']
oposition = ['AfD', 'BÜNDNIS 90/DIE GRÜNEN', 'DIE LINKE','FDP']
regierung = ['SPD', 'CDU/CSU']
Topics = ['Klimaschutz Umwelt',
          'Digitalisierung Nachhaltigkeit',
          'EU', 
          'USA Trump', 
          'Pandemie',
          'Impfung',
          'Lockdown Maßnahmen',
          'Rechtsextremist',
          'Asylbewerber Flüchtlinge Asylant Abschiebung',
          'Kriminalität Sicherheit Terror',
          'Islamist',
          'Antisemitismus', 
          'Wirtschaftskrise Ausgaben Unternehmen Wirtschaft']


Anzahl der gesamten Reden nach Partei

In [76]:
partei_reden = df[['affiliation']].value_counts()
partei_reden = partei_reden.to_frame()
partei_reden = partei_reden.reset_index()
partei_reden.columns = ['affiliation','speeches']
partei_reden = partei_reden[partei_reden['affiliation'].isin(list(parteien))]
partei_reden.plot(kind="bar",x='affiliation',y='speeches')

In [77]:
for topic in Topics: 
    result= engine.search(topic,max_results=300)
    partei_thema = result[['affiliation']].value_counts()
    partei_thema = partei_thema.to_frame()
    partei_thema = partei_thema.reset_index()
    partei_thema.columns = ['affiliation','speeches']
    partei_thema = partei_thema[partei_thema['affiliation'].isin(list(oposition))]
    partei_thema.plot(kind="bar",title = topic,x='affiliation',y='speeches')


In [81]:
for topic in Topics: 
    result= engine.search(topic,max_results=300)
    specher_partei_thema = result[['affiliation','speaker']].value_counts()[:30]
    specher_partei_thema = specher_partei_thema.to_frame()
    specher_partei_thema = specher_partei_thema.reset_index()
    specher_partei_thema.columns = ['affiliation','speaker','speeches']
    specher_partei_thema = specher_partei_thema[specher_partei_thema['affiliation'].isin(list(oposition))]
    specher_partei_thema['speaker_affiliation']=  specher_partei_thema['speaker'] +" (" + specher_partei_thema['affiliation'] +")"
    specher_partei_thema.plot(kind="barh",title = topic,x='speeches',y='speaker_affiliation',figsize=(50,50))

TypeError: bar() got an unexpected keyword argument 'figsize'

In [24]:
for topic in Topics: 
    result= engine.search(topic,max_results=300)
    partei_thema = result[['affiliation']].value_counts()
    partei_thema = partei_thema.to_frame()
    partei_thema = partei_thema.reset_index()
    partei_thema.columns = ['affiliation','speeches']
    partei_thema = partei_thema[partei_thema['affiliation'].isin(list(regierung))]
    partei_thema.plot(kind="bar",title = topic,x='affiliation',y='speeches')

In [10]:
for topic in Topics: 
    result= engine.search(topic,max_results=200)
    specher_partei_thema = result[['affiliation','speaker']].value_counts()[:40]
    specher_partei_thema = specher_partei_thema.to_frame()
    specher_partei_thema = specher_partei_thema.reset_index()
    specher_partei_thema.columns = ['affiliation','speaker','speeches']
    specher_partei_thema = specher_partei_thema[specher_partei_thema['affiliation'].isin(list(regierung))]
    specher_partei_thema['speaker_affiliation']=  specher_partei_thema['speaker'] +" (" + specher_partei_thema['affiliation'] +")"
    specher_partei_thema.plot(kind="bar",title = topic,x='speeches',y='speaker_affiliation')

In [39]:
for index,speaker,aff in  zip(df.index,df['speaker'].values,df['affiliation'].values):
    if '90' in aff:
        df.loc[index,'affiliation']= "BÜNDNIS 90/DIE GRÜNEN"
        print(aff)

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
Bündnis 90/Die Grünen
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90